# Group 14: Pathomic Fusion Boost

## 1. Exploratory Data Analysis

In [44]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import torch
import torchvision.transforms as T
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.nn import init, Parameter
from types import SimpleNamespace
from torchvision.transforms.functional import to_tensor
import torch_geometric
from torch_geometric.nn import SAGEConv, SAGPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.nn import GraphConv
from torch_geometric.data import Data, Batch
from torch.utils.data._utils.collate import default_collate

In [2]:
IMG_ROOT = "./mlcb_project/TCGA_GBMLGG_data/TCGA_GBMLGG_folder/TCGA_GBMLGG_folder/TCGA_GBMLGG/all_st"  
SPLIT_PKL   = "./mlcb_project/splits_folder/splits/gbmlgg15cv_all_st_0_0_0.pkl"
PATCH_ROOT  = "./mlcb_project/TCGA_GBMLGG_data/TCGA_GBMLGG_folder/TCGA_GBMLGG_folder/TCGA_GBMLGG/all_st_patches_512"
GRAPH_ROOT  = "./mlcb_project/TCGA_GBMLGG_data/TCGA_GBMLGG_folder/TCGA_GBMLGG_folder/TCGA_GBMLGG/all_st_cpc_img/pt_bi"    
GRAPH_PATCH = "./mlcb_project/TCGA_GBMLGG_data/TCGA_GBMLGG_folder/TCGA_GBMLGG_folder/TCGA_GBMLGG/all_st_patches_512_cpc"  
RNA_PATH    = "./mlcb_project/TCGA_GBMLGG_data/TCGA_GBMLGG_folder/TCGA_GBMLGG_folder/TCGA_GBMLGG/mRNA_Expression_Zscores_RSEM.txt"  

### Image data

In [3]:
splits_pkl = pd.read_pickle("./mlcb_project/TCGA_GBMLGG_data/splits_folder/splits/gbmlgg15cv_all_st_0_0_0.pkl")
data_pd = pd.DataFrame(splits_pkl["data_pd"])
pat2img = pd.DataFrame(list(splits_pkl["pat2img"].items()), columns=["TCGA_id", "img_fnames"])
img_fnames = pd.DataFrame(splits_pkl["img_fnames"]).rename(columns={0:"fname"})
cv_splits = pd.DataFrame(splits_pkl["cv_splits"]).T

In [4]:
data_pd.head()

,Histomolecular subtype,Histology,Grade,Molecular subtype,TCGA ID,censored,Survival months,codeletion,idh mutation,EGFR,...,16q,17p,17q,18p,18q,19p,20p,20q,21q,22q
TCGA ID,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0001,idhwt_ATC,3,2.0,0,TCGA-02-0001,1,358,0.0,0.0,1.0,...,0.000,-0.211,-0.211,0.000,0.000,0.000,-0.133,-0.133,0.591,0.000
TCGA-02-0003,idhwt_ATC,3,2.0,0,TCGA-02-0003,1,144,0.0,0.0,2.0,...,0.000,0.760,0.760,0.000,0.000,0.000,0.000,0.000,0.000,0.000
TCGA-02-0006,idhwt_ATC,3,2.0,0,TCGA-02-0006,1,558,0.0,0.0,1.0,...,0.000,0.000,0.000,-0.494,-0.494,0.000,0.476,0.476,0.496,0.000
TCGA-02-0007,idhwt_ATC,3,2.0,0,TCGA-02-0007,1,705,0.0,0.0,1.0,...,0.000,0.000,0.000,0.000,0.000,0.694,0.000,0.000,0.000,-0.939
TCGA-02-0009,idhwt_ATC,3,2.0,0,TCGA-02-0009,1,322,0.0,0.0,2.0,...,-0.862,0.000,0.000,0.000,0.000,0.738,0.000,0.000,0.000,0.000


In [5]:
pat2img.head()

,TCGA_id,img_fnames
0,TCGA-06-0166,[TCGA-06-0166-01Z-00-DX1.f0c10e84-1591-4ac1-9d...
1,TCGA-VM-A8CB,[TCGA-VM-A8CB-01Z-00-DX2.FF7BED61-5947-4261-8B...
2,TCGA-QH-A6CU,[TCGA-QH-A6CU-01Z-00-DX1.617578E5-B19E-426A-98...
3,TCGA-DB-A4XG,[TCGA-DB-A4XG-01Z-00-DX1.76683FE4-150C-44B9-99...
4,TCGA-02-0281,[TCGA-02-0281-01Z-00-DX1.6db34ca2-a4f6-4efb-ad...


In [6]:
img_fnames.head()

,fname
0,TCGA-06-0166-01Z-00-DX1.f0c10e84-1591-4ac1-9d1...
1,TCGA-VM-A8CB-01Z-00-DX2.FF7BED61-5947-4261-8B9...
2,TCGA-QH-A6CU-01Z-00-DX1.617578E5-B19E-426A-98B...
3,TCGA-DB-A4XG-01Z-00-DX1.76683FE4-150C-44B9-99C...
4,TCGA-02-0281-01Z-00-DX1.6db34ca2-a4f6-4efb-adf...


In [7]:
cv_splits.head()

,train,test
1,"{'x_patname': ['TCGA-02-0001', 'TCGA-02-0001',...","{'x_patname': ['TCGA-02-0006', 'TCGA-02-0009',..."
2,"{'x_patname': ['TCGA-02-0001', 'TCGA-02-0001',...","{'x_patname': ['TCGA-02-0007', 'TCGA-02-0010',..."
3,"{'x_patname': ['TCGA-02-0001', 'TCGA-02-0001',...","{'x_patname': ['TCGA-02-0003', 'TCGA-02-0003',..."
4,"{'x_patname': ['TCGA-02-0003', 'TCGA-02-0003',...","{'x_patname': ['TCGA-02-0001', 'TCGA-02-0001',..."
5,"{'x_patname': ['TCGA-02-0001', 'TCGA-02-0001',...","{'x_patname': ['TCGA-02-0003', 'TCGA-02-0003',..."


### RNA-seq file

In [8]:
df_rna = pd.read_csv("./mlcb_project/TCGA_GBMLGG_data/TCGA_GBMLGG_folder/TCGA_GBMLGG_folder/TCGA_GBMLGG/mRNA_Expression_Zscores_RSEM.txt", sep="\t", index_col=0)

In [9]:
df_rna.head(10)

,lgg_tcga_pan_can_atlas_2018,lgg_tcga_pan_can_atlas_2018.1,lgg_tcga_pan_can_atlas_2018.2,lgg_tcga_pan_can_atlas_2018.3,lgg_tcga_pan_can_atlas_2018.4,lgg_tcga_pan_can_atlas_2018.5,lgg_tcga_pan_can_atlas_2018.6,lgg_tcga_pan_can_atlas_2018.7,lgg_tcga_pan_can_atlas_2018.8,lgg_tcga_pan_can_atlas_2018.9,...,lgg_tcga_pan_can_atlas_2018.408,lgg_tcga_pan_can_atlas_2018.409,lgg_tcga_pan_can_atlas_2018.410,lgg_tcga_pan_can_atlas_2018.411,lgg_tcga_pan_can_atlas_2018.412,lgg_tcga_pan_can_atlas_2018.413,lgg_tcga_pan_can_atlas_2018.414,lgg_tcga_pan_can_atlas_2018.415,lgg_tcga_pan_can_atlas_2018.416,lgg_tcga_pan_can_atlas_2018.417
STUDY_ID,,,,,,,,,,,,,,,,,,,,,
SAMPLE_ID,TCGA-CS-4938-01,TCGA-CS-4941-01,TCGA-CS-4942-01,TCGA-CS-4943-01,TCGA-CS-4944-01,TCGA-CS-5393-01,TCGA-CS-5394-01,TCGA-CS-5395-01,TCGA-CS-5396-01,TCGA-CS-5397-01,...,TCGA-VV-A86M-01,TCGA-W9-A837-01,TCGA-WH-A86K-01,TCGA-WY-A858-01,TCGA-WY-A859-01,TCGA-WY-A85A-01,TCGA-WY-A85B-01,TCGA-WY-A85C-01,TCGA-WY-A85D-01,TCGA-WY-A85E-01
IDH1,-0.1856,0.3427,-0.2869,1.328,-0.6176,0.0609,0.5268,1.2162,0.4859,0.743,...,0.3353,-0.8371,-0.4375,-0.4637,-0.5765,1.4074,1.4247,-0.4127,-0.596,-0.1344
TP53,0.0446,0.3141,0.2781,2.8886,-1.1385,0.176,0.4927,0.6131,-0.6288,0.384,...,-0.0233,-0.5509,0.1164,-0.6075,-0.3013,0.6912,1.5204,0.8272,0.237,0.3866
ATRX,-1.3919,0.2507,-1.1519,-1.0866,-0.5745,-0.5212,2.7395,1.6066,2.0519,0.8454,...,-0.4125,0.7606,-1.7365,-1.0499,-1.1388,-1.0602,-0.983,-1.0044,-1.4056,-0.9676
TTN,0.2591,-0.5954,-0.3684,-0.3732,-0.7349,-0.4175,0.5335,-0.0838,-0.7622,-0.6658,...,0.2977,0.2509,-0.1428,0.5623,-0.4608,0.5839,-0.841,0.247,1.7053,-0.3733
PTEN,-0.3614,-2.8385,0.0652,-0.0894,0.0034,-0.1311,1.9415,-1.0138,-0.4419,0.2312,...,0.2333,0.6042,-0.2457,-1.0397,0.0052,-0.359,-0.0872,0.9863,-2.6312,0.8398
EGFR,-0.4755,20.372,0.352,1.7171,-0.4097,3.8277,2.5566,3.8864,-0.2695,-0.1105,...,-0.5999,0.6812,-0.7962,-0.9152,-0.9175,-0.6357,-0.0533,0.2952,-0.9668,-0.83
CIC,-1.661,1.4676,-2.1871,-0.7507,-0.9385,0.2532,-3.216,-0.2721,-2.3606,-1.341,...,-0.3393,-1.4474,0.1715,-0.7402,0.6678,0.0522,-0.4351,0.2565,-0.5274,-0.2379
MUC16,-0.2407,-0.1088,-0.2407,-0.2407,-0.2407,-0.2407,0.0151,-0.2407,-0.2407,-0.1122,...,-0.2407,-0.2407,-0.2407,-0.2407,-0.2407,-0.2407,-0.2407,0.3844,-0.2407,-0.2407


In [10]:
df_cnv = pd.read_csv("./mlcb_project/TCGA_GBMLGG_data/TCGA_GBMLGG_folder/TCGA_GBMLGG_folder/TCGA_GBMLGG/all_dataset.csv", index_col=0)

In [11]:
df_cnv.head(10)

,indexes,censored,Survival months,codeletion,idh mutation,EGFR,MDM4,MYC,BRAF,EZH2,...,16q,17p,17q,18p,18q,19p,20p,20q,21q,22q
TCGA ID,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0001,1,0,358,0.0,0.0,1.0,1.0,-1.0,1.0,1.0,...,0.000,-0.211,-0.211,0.000,0.000,0.000,-0.133,-0.133,0.591,0.000
TCGA-02-0003,2,0,144,0.0,0.0,2.0,0.0,0.0,1.0,1.0,...,0.000,0.760,0.760,0.000,0.000,0.000,0.000,0.000,0.000,0.000
TCGA-02-0006,3,0,558,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.000,0.000,0.000,-0.494,-0.494,0.000,0.476,0.476,0.496,0.000
TCGA-02-0007,4,0,705,0.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.694,0.000,0.000,0.000,-0.939
TCGA-02-0009,5,0,322,0.0,0.0,2.0,0.0,0.0,1.0,1.0,...,-0.862,0.000,0.000,0.000,0.000,0.738,0.000,0.000,0.000,0.000
TCGA-02-0010,6,0,1077,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.000,0.000,0.000,-0.473,0.000,0.000,0.000,0.000,0.000,-0.190
TCGA-02-0011,7,0,630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,-0.877,-0.460,0.000,-0.838,0.000,-0.870,0.000
TCGA-02-0014,8,0,2512,0.0,1.0,0.0,1.0,2.0,1.0,1.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
TCGA-02-0015,9,0,627,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.000,0.000,0.000,0.000,0.000,0.625,0.768,0.768,-0.387,-0.347


### Image model EDA

In [12]:
train_ids = cv_splits.loc[1, 'train']['x_patname']
test_ids  = cv_splits.loc[1, 'test']['x_patname']

In [13]:
len(train_ids),len(test_ids)

(1239, 266)

In [14]:
candidate = next(tcga for tcga in train_ids if tcga in set(pat2img['TCGA_id']))
imgs = pat2img.loc[pat2img['TCGA_id'] == candidate, 'img_fnames'].iloc[0]
one_rel = imgs[0] 
one_path = os.path.join(IMG_ROOT, one_rel)

In [15]:
one_path

'./mlcb_project/TCGA_GBMLGG_data/TCGA_GBMLGG_folder/TCGA_GBMLGG_folder/TCGA_GBMLGG/all_st/TCGA-02-0001-01Z-00-DX2.b521a862-280c-4251-ab54-5636f20605d0_1.png'

In [16]:
# check if cnn forward works for the image
img = Image.open(one_path).convert("RGB")
x = torch.from_numpy(np.array(img)).permute(2,0,1).float().unsqueeze(0)  

# sanity check
cnn = torch.hub.load('pytorch/vision', 'resnet18', pretrained=False)
cnn.fc = nn.Linear(cnn.fc.in_features, 1)

with torch.no_grad():
    risk = cnn(x).squeeze(-1) 
risk.shape, risk

Using cache found in /home/deyinhou/.cache/torch/hub/pytorch_vision_main
/home/deyinhou/.conda/envs/pathomic/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/deyinhou/.conda/envs/pathomic/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


(torch.Size([1]), tensor([-0.1866]))

In [17]:
data_pd.columns

Index(['Histomolecular subtype', 'Histology', 'Grade', 'Molecular subtype',
       'TCGA ID', 'censored', 'Survival months', 'codeletion', 'idh mutation',
       'EGFR', 'MDM4', 'MYC', 'BRAF', 'EZH2', 'MET', 'SMO', 'KIAA1549',
       'CREB3L2', 'NTRK1', 'PRCC', 'BLM', 'NTRK3', 'CRTC3', 'CDKN2A', 'CDKN2B',
       'FGFR2', 'TSHR', 'TCL1A', 'TRIP11', 'GOLGA5', 'GPHN', 'DICER1', 'TCL6',
       'EBF1', 'ITK', 'RPL22', 'CDKN2C', 'LCP1', 'RB1', 'IRF4', 'FGFR1OP',
       'MLLT4', 'MYB', 'ROS1', 'TNFAIP3', 'GOPC', 'CARD11', 'JAK2', 'STK11',
       'PTEN', '1q', '2p', '2q', '3p', '3q', '4p', '4q', '5p', '5q', '6p',
       '6q', '7p', '7q', '8p', '8q', '9p', '9q', '10p', '10q', '11p', '11q',
       '12p', '12q', '13q', '14q', '15q', '16p', '16q', '17p', '17q', '18p',
       '18q', '19p', '20p', '20q', '21q', '22q'],
      dtype='object')

In [18]:
data_pd['Survival months'].describe()

count     769.000000
mean      766.573472
std       813.549623
min         0.000000
25%       260.000000
50%       523.000000
75%       961.000000
max      6423.000000
Name: Survival months, dtype: float64

In [19]:
pat2img.columns

Index(['TCGA_id', 'img_fnames'], dtype='object')

In [20]:
# build survival for that patient
data_pd['event'] = (1 - data_pd['censored']).astype(float)

row = data_pd.loc[data_pd['TCGA ID'] == candidate].iloc[0]
t = torch.tensor([float(row['Survival months'])], dtype=torch.float32)
e = torch.tensor([float(row['event'])], dtype=torch.float32)  # 1:event, 0:censored 
t, e

(tensor([358.]), tensor([0.]))

In [21]:
# from utils.py
def CoxLoss(survtime, censor, hazard_pred, device):
    # This calculation credit to Travers Ching https://github.com/traversc/cox-nnet
    # Cox-nnet: An artificial neural network method for prognosis prediction of high-throughput omics data
    current_batch_len = len(survtime)
    R_mat = np.zeros([current_batch_len, current_batch_len], dtype=int)
    for i in range(current_batch_len):
        for j in range(current_batch_len):
            R_mat[i,j] = survtime[j] >= survtime[i]

    R_mat = torch.FloatTensor(R_mat).to(device)
    theta = hazard_pred.reshape(-1)
    exp_theta = torch.exp(theta)
    loss_cox = -torch.mean((theta - torch.log(torch.sum(exp_theta*R_mat, dim=1))) * censor)
    return loss_cox

In [51]:
# from utils.py
def CIndex(hazards, labels, survtime_all):
    concord = 0.
    total = 0.
    N_test = labels.shape[0]
    for i in range(N_test):
        if labels[i] == 1:
            for j in range(N_test):
                if survtime_all[j] > survtime_all[i]:
                    total += 1
                    if hazards[j] < hazards[i]: concord += 1
                    elif hazards[j] < hazards[i]: concord += 0.5

    return(concord/total)

def mixed_collate(batch):
    elem = batch[0]
    elem_type = type(elem)    
    transposed = zip(*batch)
    return [Batch.from_data_list(samples, []) if type(samples[0]) is torch_geometric.data.data.Data else default_collate(samples) for samples in transposed]

### Pathology model test

In [23]:
# single training
opt = torch.optim.AdamW(cnn.parameters(), lr=1e-4)
cnn.train()
opt.zero_grad()
risk = cnn(x).squeeze(-1)              
loss = CoxLoss(t, e, risk, device="cpu")
loss.backward(); opt.step()
print("Single-image Cox step OK. Loss:", float(loss))

Single-image Cox step OK. Loss: -0.0


In [24]:
data_pd['censored'].value_counts(dropna=False)

censored
1    388
0    381
Name: count, dtype: int64

In [25]:
data_pd['event'].value_counts(dropna=False)

event
0.0    388
1.0    381
Name: count, dtype: int64

In [26]:
# testing 5 patients now
id_col     = 'TCGA ID'
img_id_col = 'TCGA_id' 

# Get train IDs for current fold
fold = cv_splits.index[0]
row = cv_splits.loc[fold, 'train']
train_ids = list(row.get('x_patname', [])) if isinstance(row, dict) else list(row)

# prepare df for this fold
have_img = set(pat2img[img_id_col])
df_fold = data_pd[data_pd[id_col].isin(train_ids)].copy()
df_fold['event'] = (1 - df_fold['censored']).astype(float)  
df_fold['has_img'] = df_fold[id_col].isin(have_img)

events_df   = df_fold[(df_fold['event'] == 1.0) & (df_fold['has_img']) & df_fold['Survival months'].notna()]
censored_df = df_fold[(df_fold['event'] == 0.0) & (df_fold['has_img']) & df_fold['Survival months'].notna()]

print("With images — events:", len(events_df), "censored:", len(censored_df))

With images — events: 314 censored: 302


In [27]:
# selects patients with at least 3 events and 2 censoring for comparison
rng = 123
need_events = min(3, len(events_df))
need_cens   = min(2, len(censored_df))
pick_e = events_df.sample(need_events, random_state=rng) if need_events > 0 else events_df.head(0)
pick_c = censored_df.sample(need_cens, random_state=rng) if need_cens > 0 else censored_df.head(0)
batch_df = pd.concat([pick_e, pick_c]).reset_index(drop=True)

# # Fallback: if we still have <3 samples, just fill with more events
# if len(batch_df) < 3 and len(events_df) >= 3:
#     batch_df = events_df.sample(3, random_state=rng).reset_index(drop=True)

print("Batch target size:", len(batch_df), "; events in batch:", int(batch_df['event'].sum()))

Batch target size: 5 ; events in batch: 3


In [28]:
# image paths + labels
paths, times, events = [], [], []
for _, r in batch_df.iterrows():
    imgs = pat2img.loc[pat2img[img_id_col] == r[id_col], 'img_fnames'].iloc[0]
    rel  = imgs[0] if isinstance(imgs, (list, tuple)) else imgs
    pth  = os.path.join(IMG_ROOT, rel)
    if not os.path.exists(pth):
        continue
    paths.append(pth)
    times.append(float(r['Survival months']))
    events.append(float(r['event']))

# quickly load images 
arr = [torch.from_numpy(np.array(Image.open(p).convert("RGB"))).permute(2,0,1).float() for p in paths]
X = torch.stack(arr)                              
t = torch.tensor(times,  dtype=torch.float32)
e = torch.tensor(events, dtype=torch.float32)

print("Final batch — B:", X.shape[0], "events:", int(e.sum().item()),
      "time range:", float(t.min()), "to", float(t.max()))

# small training with cox loss
cnn.train()
opt = torch.optim.AdamW(cnn.parameters(), lr=1e-4)
opt.zero_grad()
risk = cnn(X).squeeze(-1)                        
loss = CoxLoss(t, e, risk, device="cpu")
loss.backward(); opt.step()
print("Batch Cox step OK. Loss:", float(loss))

Final batch — B: 5 events: 3 time range: 155.0 to 1891.0
Batch Cox step OK. Loss: 0.7512790560722351


In [29]:
# C-index
cidx = CIndex(risk.detach().numpy(), e.numpy(), t.numpy())
print("Batch C-index:", cidx)

Batch C-index: 0.375


In [30]:
# from data_loaders.py
class PathgraphomicFastDatasetLoader(Dataset):
    def __init__(self, opt, data, split, mode='omic'):
        """
        Args:
            X = data
            e = overall survival event
            t = overall survival in months
        """
        self.X_path = data[split]['x_path']
        self.X_grph = data[split]['x_grph']
        self.X_omic = data[split]['x_omic']
        self.e = data[split]['e']
        self.t = data[split]['t']
        self.g = data[split]['g']
        self.mode = mode

    def __getitem__(self, index):
        single_e = torch.tensor(self.e[index]).type(torch.FloatTensor)
        single_t = torch.tensor(self.t[index]).type(torch.FloatTensor)
        single_g = torch.tensor(self.g[index]).type(torch.LongTensor)

        if self.mode == "path" or self.mode == 'pathpath':
            # single_X_path = torch.tensor(self.X_path[index]).type(torch.FloatTensor).squeeze(0) # change here since the pictures are too large to convert to tensor
            single_X_path = to_tensor(Image.open(self.X_path[index]).convert("RGB")).float()
            return (single_X_path, 0, 0, single_e, single_t, single_g)
        elif self.mode == "graph" or self.mode == 'graphgraph':
            single_X_grph = torch.load(self.X_grph[index])
            return (0, single_X_grph, 0, single_e, single_t, single_g)
        elif self.mode == "omic" or self.mode == 'omicomic':
            single_X_omic = torch.tensor(self.X_omic[index]).type(torch.FloatTensor)
            return (0, 0, single_X_omic, single_e, single_t, single_g)
        elif self.mode == "pathomic":
            # single_X_path = torch.tensor(self.X_path[index]).type(torch.FloatTensor).squeeze(0)
            single_X_path = to_tensor(Image.open(self.X_path[index]).convert("RGB")).float()
            single_X_omic = torch.tensor(self.X_omic[index]).type(torch.FloatTensor)
            return (single_X_path, 0, single_X_omic, single_e, single_t, single_g)
        elif self.mode == "graphomic":
            single_X_grph = torch.load(self.X_grph[index])
            single_X_omic = torch.tensor(self.X_omic[index]).type(torch.FloatTensor)
            return (0, single_X_grph, single_X_omic, single_e, single_t, single_g)
        elif self.mode == "pathgraph":
            # single_X_path = torch.tensor(self.X_path[index]).type(torch.FloatTensor).squeeze(0)
            single_X_path = to_tensor(Image.open(self.X_path[index]).convert("RGB")).float()
            single_X_grph = torch.load(self.X_grph[index])
            return (single_X_path, single_X_grph, 0, single_e, single_t, single_g)
        elif self.mode == "pathgraphomic":
            # single_X_path = torch.tensor(self.X_path[index]).type(torch.FloatTensor).squeeze(0)
            single_X_path = to_tensor(Image.open(self.X_path[index]).convert("RGB")).float()
            single_X_grph = torch.load(self.X_grph[index])
            single_X_omic = torch.tensor(self.X_omic[index]).type(torch.FloatTensor)
            return (single_X_path, single_X_grph, single_X_omic, single_e, single_t, single_g)

    def __len__(self):
        return len(self.X_path)

In [31]:
use_patches = False  # set True if split has all_st_patches_512 in its name (and *_cpc) 
IMG_DIR   = PATCH_ROOT if use_patches else IMG_ROOT
GRAPH_DIR = GRAPH_PATCH if use_patches else GRAPH_ROOT

In [32]:
train_ids = list(cv_splits.loc[fold, 'train']['x_patname'])
test_ids  = list(cv_splits.loc[fold, 'test']['x_patname'])

ID_COL, TIME_COL, EVENT_COL = 'TCGA ID', 'Survival months', 'event'

In [33]:
meta = data_pd.set_index(ID_COL)
img_map = dict(pat2img[['TCGA_id','img_fnames']].values)

In [34]:
def build_split(ids):
    xp, e, t, g = [], [], [], []
    for pid in ids:
        imgs = img_map.get(pid); 
        if not imgs or pid not in meta.index: continue
        rel = imgs[0] if isinstance(imgs, (list,tuple)) else imgs
        full = os.path.join(IMG_ROOT, rel)
        if not os.path.exists(full): continue
        ti, ev = meta.loc[pid, TIME_COL], meta.loc[pid, EVENT_COL]
        if pd.isna(ti) or pd.isna(ev): continue
        xp.append(full); t.append(float(ti)); e.append(float(ev)); g.append(np.nan)
    return dict(x_path=xp, x_grph=[None]*len(xp), x_omic=[None]*len(xp),
                e=np.array(e,float), t=np.array(t,float), g=np.array(g,float))

data = {'train': build_split(train_ids), 'test': build_split(test_ids)}

In [35]:
opt = SimpleNamespace()
train_ds = PathgraphomicFastDatasetLoader(opt=opt, data=data, split='train', mode='path')
test_ds  = PathgraphomicFastDatasetLoader(opt=opt, data=data, split='test',  mode='path')
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True,  num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=8, shuffle=False, num_workers=2, pin_memory=True)

In [36]:
# copy from networks.py

############
# Path Model
############
model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}

class PathNet(nn.Module):

    def __init__(self, features, path_dim=32, act=None, num_classes=1):
        super(PathNet, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 1024),
            nn.ReLU(True),
            nn.Dropout(0.25),
            nn.Linear(1024, 1024),
            nn.ReLU(True),
            nn.Dropout(0.25),
            nn.Linear(1024, path_dim),
            nn.ReLU(True),
            nn.Dropout(0.05)
        )

        self.linear = nn.Linear(path_dim, num_classes)
        self.act = act

        self.output_range = Parameter(torch.FloatTensor([6]), requires_grad=False)
        self.output_shift = Parameter(torch.FloatTensor([-3]), requires_grad=False)

        dfs_freeze(self.features)

    def forward(self, **kwargs):
        x = kwargs['x_path']
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        features = self.classifier(x)
        hazard = self.linear(features)

        if self.act is not None:
            hazard = self.act(hazard)

            if isinstance(self.act, nn.Sigmoid):
                hazard = hazard * self.output_range + self.output_shift

        return features, hazard


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


# change model if needed
def get_vgg(arch='vgg11_bn', cfg='E', act=None, batch_norm=True, label_dim=1, pretrained=True, progress=True, **kwargs):
    model = PathNet(make_layers(cfgs[cfg], batch_norm=batch_norm), act=act, num_classes=label_dim, **kwargs)
    
    if pretrained:
        pretrained_dict = load_state_dict_from_url(model_urls[arch], progress=progress)

        for key in list(pretrained_dict.keys()):
            if 'classifier' in key: pretrained_dict.pop(key)

        model.load_state_dict(pretrained_dict, strict=False)
        print("Initializing Path Weights")

    return model

In [37]:
# from utils.py
def dfs_freeze(model):
    for name, child in model.named_children():
        for param in child.parameters():
            param.requires_grad = False
        dfs_freeze(child)

In [38]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# change model if needed
model = get_vgg(arch='vgg11_bn', cfg='E', act=None, batch_norm=True,
                label_dim=1, pretrained=False,  # set True only if download is OK
                path_dim=32).to(device)
optm = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
CIDX = globals().get("CIndex_lifeline", globals().get("CIndex"))

In [39]:
unc = [pid for pid in train_ids if pid in meta.index and meta.loc[pid,'event']==1]
unc.sort(key=lambda pid: meta.loc[pid,'Survival months'])
seed = []
for pid in unc:
    if not seed or meta.loc[pid,'Survival months'] != meta.loc[seed[0],'Survival months']:
        seed.append(pid)
    if len(seed)==2: break

rest = [pid for pid in train_ids if pid not in seed and pid in meta.index]
ids5 = seed + rest[:max(0, 5-len(seed))]

data = {'train': build_split(ids5), 'test': build_split(test_ids[:5])}
train_ds = PathgraphomicFastDatasetLoader(opt=None, data=data, split='train', mode='path')
train_loader = DataLoader(train_ds, batch_size=len(ids5), shuffle=False, num_workers=0)

In [41]:
def run_epoch(loader, train=True):
    model.train() if train else model.eval()
    tot, n = 0.0, 0; rs, ts, es = [], [], []
    for batch in loader:
        x, t, e = batch[0], batch[4], batch[3]
        x,t,e = x.to(device).float(), t.to(device).float(), e.to(device).float()
        if train: optm.zero_grad()
        with torch.set_grad_enabled(train):
            r = model(x_path=x)[1].squeeze(-1)
            loss = CoxLoss(t,e,r,device=device)
            if train: loss.backward(); optm.step()
        bs = x.size(0); tot += float(loss)*bs; n += bs
        rs.append(r.detach().cpu()); ts.append(t.cpu()); es.append(e.cpu())
    import torch as _t
    r,tt,ee = _t.cat(rs).numpy(), _t.cat(ts).numpy(), _t.cat(es).numpy()
    return tot/max(n,1), CIDX(r, ee, tt)

for ep in range(1, 3):
    tr_l, tr_c = run_epoch(train_loader, True)
    te_l, te_c = run_epoch(test_loader,  False)
    print(f"Ep{ep:02d}  train {tr_l:.4f} C {tr_c:.3f}  |  test {te_l:.4f} C {te_c:.3f}")

Ep01  train 0.6150 C 0.286  |  test 0.6292 C 0.276
Ep02  train 0.5204 C 1.000  |  test 0.6292 C 0.294


### Graph model test

In [82]:
# Import from networks.py
############
# Graph Model
############
# class NormalizeFeaturesV2(object):
#     r"""Column-normalizes node features to sum-up to one."""

#     def __call__(self, data):
#         data.x = data.x / data.x.max(0, keepdim=True)[0]#.type(torch.cuda.FloatTensor)
#         return data

#     def __repr__(self):
#         return '{}()'.format(self.__class__.__name__)

# class NormalizeFeaturesV2(object):
#     r"""Column-normalizes node features to sum-up to one."""

#     def __call__(self, data):
#         data.x[:, :12] = data.x[:, :12] / data.x[:, :12].max(0, keepdim=True)[0]
#         data.x = data.x.type(torch.cuda.FloatTensor)
#         return data

#     def __repr__(self):
#         return '{}()'.format(self.__class__.__name__)


# class NormalizeEdgesV2(object):
#     r"""Column-normalizes node features to sum-up to one."""

#     def __call__(self, data):
#         data.edge_attr = data.edge_attr.type(torch.cuda.FloatTensor)
#         data.edge_attr = data.edge_attr / data.edge_attr.max(0, keepdim=True)[0]#.type(torch.cuda.FloatTensor)
#         return data

#     def __repr__(self):
#         return '{}()'.format(self.__class__.__name__)

class NormalizeFeaturesV2(object):
    def __call__(self, data):
        denom = data.x.max(0, keepdim=True)[0].clamp_min(1e-12)
        data.x = data.x / denom
        return data

class NormalizeEdgesV2(object):
    def __call__(self, data):
        if getattr(data, "edge_attr", None) is not None:
            denom = data.edge_attr.max(0, keepdim=True)[0].clamp_min(1e-12)
            data.edge_attr = data.edge_attr / denom
        return data


class GraphNet(torch.nn.Module):
    def __init__(self, features=1036, nhid=128, grph_dim=32, nonlinearity=torch.tanh, 
        dropout_rate=0.25, GNN='GCN', use_edges=0, pooling_ratio=0.20, act=None, label_dim=1, init_max=True):
        super(GraphNet, self).__init__()

        self.dropout_rate = dropout_rate
        self.use_edges = use_edges
        self.act = act

        self.conv1 = SAGEConv(features, nhid)
        self.pool1 = SAGPooling(nhid, ratio=pooling_ratio, GNN=GraphConv)# gnn=GNN)#, nonlinearity=nonlinearity)
        self.conv2 = SAGEConv(nhid, nhid)
        self.pool2 = SAGPooling(nhid, ratio=pooling_ratio, GNN=GraphConv)# gnn=GNN)#, nonlinearity=nonlinearity)
        self.conv3 = SAGEConv(nhid, nhid)
        self.pool3 = SAGPooling(nhid, ratio=pooling_ratio, GNN=GraphConv)# gnn=GNN)#, nonlinearity=nonlinearity)

        self.lin1 = torch.nn.Linear(nhid*2, nhid)
        self.lin2 = torch.nn.Linear(nhid, grph_dim)
        self.lin3 = torch.nn.Linear(grph_dim, label_dim)

        self.output_range = Parameter(torch.FloatTensor([6]), requires_grad=False)
        self.output_shift = Parameter(torch.FloatTensor([-3]), requires_grad=False)

        if init_max: 
            init_max_weights(self)
            print("Initialzing with Max")

    def forward(self, **kwargs):
        data = kwargs['x_grph']
        data = NormalizeFeaturesV2()(data)
        data = NormalizeEdgesV2()(data)
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        #x, edge_index, edge_attr, batch = data.x.type(torch.cuda.FloatTensor), data.edge_index.type(torch.cuda.LongTensor), data.edge_attr.type(torch.cuda.FloatTensor), data.batch
        x = F.relu(self.conv1(x, edge_index))
        # x, edge_index, edge_attr, batch, _ = self.pool1(x, edge_index, edge_attr, batch)
        x, edge_index, edge_attr, batch, *_ = self.pool1(x, edge_index, edge_attr, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        # x, edge_index, edge_attr, batch, _ = self.pool2(x, edge_index, edge_attr, batch)
        x, edge_index, edge_attr, batch, *_ = self.pool2(x, edge_index, edge_attr, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))
        # x, edge_index, edge_attr, batch, _ = self.pool3(x, edge_index, edge_attr, batch)
        x, edge_index, edge_attr, batch, *_ = self.pool3(x, edge_index, edge_attr, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3 

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        features = F.relu(self.lin2(x))
        out = self.lin3(features)
        if self.act is not None:
            out = self.act(out)

            if isinstance(self.act, nn.Sigmoid):
                out = out * self.output_range + self.output_shift

        return features, out

In [61]:
def img_to_graph_path(img_rel):
    base = os.path.splitext(img_rel)[0] + ".pt"
    return os.path.join(GRAPH_ROOT, base)

def build_split_graph(ids):
    xg, e, t, g = [], [], [], []
    for pid in ids:
        imgs = img_map.get(pid)
        if not imgs or pid not in meta.index: continue
        rel = imgs[0] if isinstance(imgs,(list,tuple)) else imgs
        gpath = img_to_graph_path(rel)
        if not os.path.exists(gpath): continue
        ti, ev = meta.loc[pid, TIME_COL], meta.loc[pid, EVENT_COL]
        if pd.isna(ti) or pd.isna(ev): continue
        xg.append(gpath); t.append(float(ti)); e.append(float(ev)); g.append(np.nan)
    N = len(xg)
    return dict(x_path=[0]*N, x_grph=xg, x_omic=[0]*N,
                e=np.array(e,float), t=np.array(t,float), g=np.array(g,float))

data = {'train': build_split_graph(train_ids), 'test': build_split_graph(test_ids)}

In [77]:
# used when batch size is larger than 1
def mixed_collate(batch):
    cols = list(zip(*batch)); out = []
    for i, col in enumerate(cols):
        if i == 1 and col[0] not in (0, None):  # graph column
            gs = []
            for g in col:
                if isinstance(g, (str, bytes)):
                    g = torch.load(g, map_location="cpu")
                # ALWAYS rebuild into a fresh Data (avoids legacy _store crash)
                d = g if isinstance(g, dict) else {k: v for k, v in vars(g).items() if not str(k).startswith("_")}
                g = Data(**d)
                # drop out-of-range edges (some legacy graphs mismatch)
                if getattr(g, "edge_index", None) is not None and getattr(g, "x", None) is not None:
                    n = g.x.size(0)
                    m = (g.edge_index[0] < n) & (g.edge_index[1] < n)
                    g.edge_index = g.edge_index[:, m]
                    if getattr(g, "edge_attr", None) is not None:
                        g.edge_attr = g.edge_attr[m]
                gs.append(g)
            out.append(Batch.from_data_list(gs))
        else:
            out.append(default_collate(list(col)))
    return out


In [78]:
train_ds = PathgraphomicFastDatasetLoader(opt=None, data=data, split='train', mode='graph')
test_ds  = PathgraphomicFastDatasetLoader(opt=None, data=data, split='test',  mode='graph')

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True,  num_workers=0, collate_fn=mixed_collate)# collate_fn=lambda b: b[0])
test_loader  = DataLoader(test_ds,  batch_size=8, shuffle=False, num_workers=0, collate_fn=mixed_collate)# collate_fn=lambda b: b[0])

In [81]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = GraphNet(grph_dim=32, label_dim=1, dropout_rate=0.25, init_max=False).to(device)
optm = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
CIDX = globals().get("CIndex_lifeline", globals().get("CIndex"))

def run_epoch(loader, train=True):
    model.train() if train else model.eval()
    tot, n = 0.0, 0; rs, ts, es = [], [], []
    for batch in loader:  # (x_path, x_grph, x_omic, e, t, g)
        xg, e, t = batch[1], batch[3].float(), batch[4].float()

        # If it's already a Batch (from mixed_collate), keep it. Otherwise rebuild a Data.
        if not isinstance(xg, Batch):
            d = xg if isinstance(xg, dict) else getattr(xg, "__dict__", {})
            xg = Data(**{k: v for k, v in d.items() if not str(k).startswith("_")})
            # fix edge index > num_nodes by padding x
            if getattr(xg, "x", None) is not None and getattr(xg, "edge_index", None) is not None and xg.edge_index.numel() > 0:
                need_n = int(xg.edge_index.max().item()) + 1
                have_n = xg.x.size(0)
                if need_n > have_n:
                    xg.x = F.pad(xg.x, (0, 0, 0, need_n - have_n))

        xg, e, t = xg.to(device), e.to(device), t.to(device)

        if train: optm.zero_grad()
        with torch.set_grad_enabled(train):
            r = model(x_grph=xg)[1].squeeze(-1)
            t = t.view(-1); e = e.view(-1); r = r.view(-1)
            loss = CoxLoss(t, e, r, device=device)
            if train: loss.backward(); optm.step()

        bs = e.size(0); tot += float(loss) * bs; n += bs
        rs.append(r.detach().cpu()); ts.append(t.cpu()); es.append(e.cpu())

    import torch as _t
    r, tt, ee = _t.cat(rs).numpy(), _t.cat(ts).numpy(), _t.cat(es).numpy()
    return tot / max(n, 1), CIDX(r, ee, tt)

for ep in range(1, 2+1):
    tr_l, tr_c = run_epoch(train_loader, True)
    te_l, te_c = run_epoch(test_loader,  False)
    print(f"Ep{ep:02d} | train {tr_l:.4f} C {tr_c:.3f} | test {te_l:.4f} C {te_c:.3f}")

Ep01 | train 0.6611 C 0.561 | test 0.6286 C 0.600
Ep02 | train 0.6388 C 0.593 | test 0.6418 C 0.623


### Omic model test

In [116]:
############
# Omic Model
############
class MaxNet(nn.Module):
    def __init__(self, input_dim=80, omic_dim=32, dropout_rate=0.25, act=None, label_dim=1, init_max=True):
        super(MaxNet, self).__init__()
        hidden = [64, 48, 32, 32]
        self.act = act

        encoder1 = nn.Sequential(
            nn.Linear(input_dim, hidden[0]),
            nn.ELU(),
            nn.AlphaDropout(p=dropout_rate, inplace=False))
        
        encoder2 = nn.Sequential(
            nn.Linear(hidden[0], hidden[1]),
            nn.ELU(),
            nn.AlphaDropout(p=dropout_rate, inplace=False))
        
        encoder3 = nn.Sequential(
            nn.Linear(hidden[1], hidden[2]),
            nn.ELU(),
            nn.AlphaDropout(p=dropout_rate, inplace=False))

        encoder4 = nn.Sequential(
            nn.Linear(hidden[2], omic_dim),
            nn.ELU(),
            nn.AlphaDropout(p=dropout_rate, inplace=False))
        
        self.encoder = nn.Sequential(encoder1, encoder2, encoder3, encoder4)
        self.classifier = nn.Sequential(nn.Linear(omic_dim, label_dim))

        if init_max: init_max_weights(self)

        self.output_range = Parameter(torch.FloatTensor([6]), requires_grad=False)
        self.output_shift = Parameter(torch.FloatTensor([-3]), requires_grad=False)

    def forward(self, **kwargs):
        x = kwargs['x_omic']
        features = self.encoder(x)
        out = self.classifier(features)
        if self.act is not None:
            out = self.act(out)

            if isinstance(self.act, nn.Sigmoid):
                out = out * self.output_range + self.output_shift

        return features, out

In [83]:
data_pd.columns

Index(['Histomolecular subtype', 'Histology', 'Grade', 'Molecular subtype',
       'TCGA ID', 'censored', 'Survival months', 'codeletion', 'idh mutation',
       'EGFR', 'MDM4', 'MYC', 'BRAF', 'EZH2', 'MET', 'SMO', 'KIAA1549',
       'CREB3L2', 'NTRK1', 'PRCC', 'BLM', 'NTRK3', 'CRTC3', 'CDKN2A', 'CDKN2B',
       'FGFR2', 'TSHR', 'TCL1A', 'TRIP11', 'GOLGA5', 'GPHN', 'DICER1', 'TCL6',
       'EBF1', 'ITK', 'RPL22', 'CDKN2C', 'LCP1', 'RB1', 'IRF4', 'FGFR1OP',
       'MLLT4', 'MYB', 'ROS1', 'TNFAIP3', 'GOPC', 'CARD11', 'JAK2', 'STK11',
       'PTEN', '1q', '2p', '2q', '3p', '3q', '4p', '4q', '5p', '5q', '6p',
       '6q', '7p', '7q', '8p', '8q', '9p', '9q', '10p', '10q', '11p', '11q',
       '12p', '12q', '13q', '14q', '15q', '16p', '16q', '17p', '17q', '18p',
       '18q', '19p', '20p', '20q', '21q', '22q', 'event'],
      dtype='object')

In [101]:
META = {'Histomolecular subtype','Histology','Grade','Molecular subtype',
        'TCGA ID','censored','Survival months'}
OMIC_COLS = [c for c in data_pd.columns if c not in META]

In [112]:
train_ids = list(cv_splits.at[fold, 'train']['x_patname'])
test_ids  = list(cv_splits.at[fold,  'test']['x_patname'])

df_tr = data_pd.set_index('TCGA ID').loc[train_ids]
df_te = data_pd.set_index('TCGA ID').loc[test_ids]

def xtet(df):
    X = df[OMIC_COLS].apply(pd.to_numeric, errors='coerce'); X = X.fillna(X.mean()).values.astype('float32')
    e = (1 - df['censored'].astype(float)).values.astype('float32')
    t = df['Survival months'].astype(float).values.astype('float32')
    return X, e, t

Xtr, etr, ttr = xtet(df_tr)
Xte, ete, tte = xtet(df_te)

In [114]:
class OmicOnlyDataset(Dataset):
    def __init__(self, X,e,t):
        self.X = torch.from_numpy(X); self.e = torch.from_numpy(e); self.t = torch.from_numpy(t)
    def __len__(self): return self.X.shape[0]
    def __getitem__(self,i): return (0,0,self.X[i], self.e[i], self.t[i], torch.tensor(0))

train_loader = DataLoader(OmicOnlyDataset(Xtr,etr,ttr), batch_size=16, shuffle=True,  num_workers=0)
test_loader  = DataLoader(OmicOnlyDataset(Xte,ete,tte), batch_size=16, shuffle=False, num_workers=0)

In [117]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MaxNet(input_dim=len(OMIC_COLS), omic_dim=32, dropout_rate=0.25, act=None, label_dim=1, init_max=False).to(device)
optm = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
CIDX = globals().get("CIndex_lifeline", globals().get("CIndex"))  # whichever you already defined

def run_epoch_omic(loader, train=True):
    model.train() if train else model.eval()
    tot, n = 0.0, 0; rs, ts, es = [], [], []
    for batch in loader:
        xo, e, t = batch[2].to(device), batch[3].to(device), batch[4].to(device)
        if train: optm.zero_grad()
        with torch.set_grad_enabled(train):
            r = model(x_omic=xo)[1].squeeze(-1)
            t, e, r = t.view(-1), e.view(-1), r.view(-1)
            loss = CoxLoss(t, e, r, device=device)
            if train: loss.backward(); optm.step()
        bs = e.size(0); tot += float(loss)*bs; n += bs
        rs.append(r.detach().cpu()); ts.append(t.cpu()); es.append(e.cpu())
    import torch as _t
    r, tt, ee = _t.cat(rs).numpy(), _t.cat(ts).numpy(), _t.cat(es).numpy()
    return tot/max(n,1), CIDX(r, ee, tt)

# 6) Sanity run
for ep in range(1, 6):
    tr_l, tr_c = run_epoch_omic(train_loader, True)
    te_l, te_c = run_epoch_omic(test_loader,  False)
    print(f"Ep{ep:02d} | train {tr_l:.4f} C {tr_c:.3f} | test {te_l:.4f} C {te_c:.3f}")

Ep01 | train 1.0044 C 0.532 | test 0.8450 C 0.621
Ep02 | train 0.9965 C 0.511 | test 0.8445 C 0.710
Ep03 | train 1.0167 C 0.479 | test 0.8442 C 0.709
Ep04 | train 1.0081 C 0.495 | test 0.8439 C 0.709
Ep05 | train 0.9926 C 0.493 | test 0.8437 C 0.708
